In [21]:
import ipywidgets as widgets
import traitlets # import CInt, Unicode

class D3SankeyWidget(widgets.DOMWidget):
    # the name of the requirejs module (no .js!)
    _view_module = traitlets.Unicode(
        'd3sankeyview').tag(sync=True)
    # the name of the Backbone.View subclass to be used
    _view_name = traitlets.Unicode(
        'D3SankeyView').tag(sync=True) 
    
    # the name of the CSS file to load with this widget
    _view_style = traitlets.Unicode(
        'nbextensions/ipythond3sankey/css/widget_d3sankey').tag(sync=True)
    
    # the actual value: lists of nodes and links
    nodes = traitlets.List().tag(sync=True)
    links = traitlets.List().tag(sync=True)
    node_link_json = traitlets.Unicode('{}').tag(sync=True)

    selected_json = traitlets.Unicode().tag(sync=True)
    
    # margins & size
    margin_top = traitlets.Float(1).tag(sync=True)
    margin_right = traitlets.Float(1).tag(sync=True)
    margin_bottom = traitlets.Float(6).tag(sync=True)
    margin_left = traitlets.Float(1).tag(sync=True)
    width = traitlets.Float(960).tag(sync=True)
    height = traitlets.Float(500).tag(sync=True)
    
    unit = traitlets.Unicode('').tag(sync=True)


In [2]:
from IPython.display import HTML
HTML('''
    <style>
      .node rect {
        fill-opacity: .9;
        shape-rendering: crispEdges;
        stroke-width: 0;
      }
      .node text {
        text-shadow: 0 1px 0 #fff;
      }
      .link {
        fill: none;
        stroke: #000;
        stroke-opacity: .2;
      }
    </style>
''')
    

In [3]:
%%javascript
require.config({
  paths: {
      d3: '/custom/d3sankey/js/d3',
      jquery: '/custom/d3sankey/js/jquery',
      underscore: '/custom/d3sankey/js/underscore',
      sankey: '/custom/d3sankey/js/sankey',
      d3_chart: '/custom/d3sankey/js/d3.chart',
      d3_chart_sankey: '/custom/d3sankey/js/d3.chart.sankey',
  },
  shim: {
    'sankey': { deps:  ['d3']},
    'd3_chart': { deps:  ['d3']},
    'd3_chart_sankey': { deps:  ['d3_chart']}
  },
});

/* //cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min
  'jquery',
  'underscore',

  // ipython API
  'widgets/js/widget',

  // local imports
  './utils',
  './d3.min',
  './sankey',
*/
      

<IPython.core.display.Javascript object>

In [22]:
%%javascript
// widget_d3sankey.js @ /home/duan/anaconda3/lib/python3.5/site-packages/ipythond3sankey/static/ipythond3sankey/js
requirejs.undef('d3sankeyview');
var global_json_data = {'test':'testduan'}
var rect_selected = {}
var rect_color = {}

define('d3sankeyview', ["jupyter-js-widgets"], function(widgets) {
  var D3SankeyView = widgets.DOMWidgetView.extend({
    // namespace your CSS so that you don't break other people's stuff
    className: 'D3SankeyView',

    //loadCss: utils.loadCss,

    // Initialize DOM, etc. called once per view creation,
    // i.e. `display(widget)`
    render: function() {

      // add a stylesheet, if defined in `_view_style`
      //this.loadCss();
      var thatthis = this;
      var graph = JSON.parse(this.model.get('node_link_json'));
      console.log('started drawing for json: ' );
      console.log( graph );

      require(['d3','jquery', 'underscore', 'sankey','d3_chart','d3_chart_sankey'], 
              function(d3,$,_, sankey,d3_chart, d3_chart_sankey) {

      
      console.log('started margin_top 2.' + thatthis.model.get('margin_top'));
      // console.log('started margin_top 3.' + this.model.get('margin_top'));
      // setup
      // XXX margins, width and height not updated
      var margin = {top:    thatthis.model.get('margin_top'),
                    right:  thatthis.model.get('margin_right'),
                    bottom: thatthis.model.get('margin_bottom'),
                    left:   thatthis.model.get('margin_left')},
          width = thatthis.model.get('width') - margin.left - margin.right,
          height = thatthis.model.get('height') - margin.top - margin.bottom;

      thatthis.width = width;
      thatthis.height = height;

      var formatNumber = d3.format(",.0f"),
          unit = thatthis.model.get('unit');

      var format = function(d) { return formatNumber(d) + " " + unit; };
      var color = d3.scale.category20();

      var svg = d3.select(thatthis.$el[0]).append("svg")
          .attr("width", width + margin.left + margin.right)
          .attr("height", height + margin.top + margin.bottom)
          .append("g")
          .attr("transform",
                "translate(" + margin.left + "," + margin.top + ")");
    // Set the sankey diagram properties
    var sankey = d3.sankey()
        .nodeWidth(36)
        .nodePadding(40)
        .size([width, height]);

var path = sankey.link();



  //set up graph in same style as original example but empty
 console.log(JSON.stringify(graph));

  sankey
    .nodes(graph.nodes)
    .links(graph.links)
    .layout(32);
 console.log( 'add in the links' );

// add in the links
  var link = svg.append("g").selectAll(".link")
      .data(graph.links)
    .enter().append("path")
      .attr("class", "link")
      .attr("d", path)
      .style("stroke-width", function(d) { return Math.max(1, d.dy); })
      .sort(function(a, b) { return b.dy - a.dy; });

// add the link titles
  link.append("title")
        .text(function(d) {
    		return d.source.name + " → " + 
                d.target.name + "\n" + format(d.value); });
 console.log( 'add in the nodes' );

          
// add in the nodes
  var node = svg.append("g").selectAll(".node")
      .data(graph.nodes)
    .enter().append("g")
      .attr("class", "node")
      .attr("transform", function(d) { 
		  return "translate(" + d.x + "," + d.y + ")"; })
    .call(d3.behavior.drag()
      .origin(function(d) { return d; })
      .on("dragstart", function() { 
		  this.parentNode.appendChild(this); })
      .on("drag", dragmove));

// add the rectangles for the nodes
  node.append("rect")
      .attr("height", function(d) { return d.dy; })
      .attr("width", sankey.nodeWidth())
      .style("fill", function(d) { 
		  return d.color = color(d.name.replace(/ .*/, "")); })
      .style("stroke", function(d) { 
		  return d3.rgb(d.color).darker(2); })
    .append("title")
      .text(function(d) { 
		  return d.name + "\n" + format(d.value); });

// add in the title for the nodes
  node.append("text")
      .attr("x", -6)
      .attr("y", function(d) { return d.dy / 2; })
      .attr("dy", ".35em")
      .attr("text-anchor", "end")
      .attr("transform", null)
      .text(function(d) { return d.name; })
    .filter(function(d) { return d.x < width / 2; })
      .attr("x", 6 + sankey.nodeWidth())
      .attr("text-anchor", "start");

// the function for moving the nodes
  function dragmove(d) {
    d3.select(this).attr("transform", 
        "translate(" + d.x + "," + (
                d.y = Math.max(0, Math.min(height - d.dy, d3.event.y))
            ) + ")");
    sankey.relayout();
    link.attr("d", path);
  }
           
d3.selectAll("rect")
.on("click",function(d,i){
        // alert("clicked!"+d.value+":"+ d.name);
        console.log("clicked!"+d.value+":"+ d.name);
        console.log('current fill core is: ');
        console.log(d3.select(this).style('fill'));
        if (rect_selected[d.name]  === undefined) {
            console.log('first time click');
            rect_selected[d.name] ='y';
            rect_color[d.name] = d3.select(this).style('fill');
            d3.select(this).style('fill','red');
        } else if (rect_selected[d.name]  == 'y') { 
            console.log('revert back to original color');
            rect_selected[d.name] ='n';
            d3.select(this).style('fill',rect_color[d.name]);
        } else{
            console.log('not first time click, but selected');
            rect_selected[d.name] ='y';
            rect_color[d.name] = d3.select(this).style('fill');
            d3.select(this).style('fill','red');
        }
        console.log(JSON.stringify(rect_selected));
        console.log( (rect_selected));
    
        console.log('current selected_json is: ');
        console.log( thatthis.model.get('selected_json'));
        thatthis.model.set('selected_json',JSON.stringify(rect_selected));

        thatthis.touch();
        console.log( thatthis.model.get('selected_json'));
        
    
    })  
//console.log('setup trigger is done 3');
//console.log(d3.selectAll('rect'));

          
      return thatthis;
 
      });
    }, // /render

    // Do things that are updated every time `this.model` is changed...
    // from the front-end or backend.
    update: function(options) {

      console.log('update function called');
         
    }, // /update


    // Tell Backbone to listen to events (none for now)
    events: {
    },

  }); // /extend

  // The requirej6s namespace.
  return {
    D3SankeyView: D3SankeyView
  };
});

console.log(global_json_data)

<IPython.core.display.Javascript object>

In [5]:
pydata = {"nodes":[{"name":"Barry"},{"name":"Elvis"},{"name":"Frodo"},
                   {"name":"Sarah"},{"name":"Alice"}],
          "links":[{"source":0,"target":1,"value":2},{"source":2,"target":1,"value":2},
                   {"source":2,"target":3,"value":2},{"source":0,"target":4,"value":2},
                   {"source":1,"target":3,"value":2},{"source":1,"target":4,"value":2},
                   {"source":3,"target":4,"value":4}]}
import json
json_string = json.dumps(pydata)

d3sankeyExample = D3SankeyWidget(
    node_link_json = json_string,
    height = 300,
    width = 600
)

In [23]:
d3sankeyExample

In [7]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
display(button)

def on_button_clicked(b):
    print(d3sankeyExample.selected_json)

button.on_click(on_button_clicked)

{"Frodo":"y","Barry":"y","Sarah":"n"}
{"Frodo":"y","Barry":"y","Sarah":"n"}
{"Frodo":"y","Barry":"y","Sarah":"n"}
{"Frodo":"y","Barry":"y","Sarah":"n","Elvis":"y"}
{"Frodo":"y","Barry":"n","Sarah":"n","Elvis":"y"}
{"Frodo":"y","Barry":"n","Sarah":"n","Elvis":"y"}
{"Frodo":"y","Barry":"y","Sarah":"n","Elvis":"n"}


In [24]:
print(d3sankeyExample.selected_json)

{"Frodo":"y","Barry":"y"}
